In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
import numpy as np

In [14]:
IgA = [
    [83, 85],
    [84, 85, 85, 86, 86, 87],
    [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    [89, 90, 90, 91],
    [90, 92]]
N = 25
alpha = 0.05

### Вспомогательные функции

In [15]:
def getTSS (y):
    return sum ([ (y_i - np.mean(y))**2 for y_i in y])

def getRegression (psi, Y):
    F = psi.T @ psi
    F_inverse = np.linalg.inv (F)
    
    betta = (F_inverse @ psi.T) @ Y
    
    e = Y - (psi @ betta)
    RSS = e.T @ e
    TSS = getTSS (Y)
    R = (TSS - RSS) / TSS
    
    return betta, RSS, TSS, R, e

### a) Определить влияние возраста на содержание иммуноглобулина в крови

In [16]:
psi = np.zeros ((N, len (IgA)))
etta = np.zeros (N)

it = 0
for i in range (len (IgA)):
    for j in range (len (IgA[i])):
        etta [it] = IgA [i][j]
        psi [it][i] = 1
        it += 1

In [18]:
F = psi.T @ psi
F_inverse = np.linalg.inv (F)
regression = getRegression (psi, etta)

print ("Коэффициенты регрессии:", regression [0])

RSS = regression [1]

for i in range (5):
    delta_up = regression[0][i] * np.sqrt (N - 5)
    delta_down = np.sqrt (RSS * F_inverse[i][i])
    
    delta = delta_up / delta_down
    
    p_value = 2 * (1 - stats.t.cdf (np.abs(delta), N - 5))
    print ("Для коэффициента {i} p-value = {p_value}".format (i = i, p_value = p_value))
    if (p_value > alpha):
        print ("Не значим.")
    else:
        print ("Значим.")

Коэффициенты регрессии: [84.         85.5        87.81818182 90.         91.        ]
Для коэффициента 0 p-value = 0.0
Значим.
Для коэффициента 1 p-value = 0.0
Значим.
Для коэффициента 2 p-value = 0.0
Значим.
Для коэффициента 3 p-value = 0.0
Значим.
Для коэффициента 4 p-value = 0.0
Значим.


In [20]:
R = regression [3]
print ("Коэффициент детерминации:", R)

delta = (R / (1 - R)) * (N - 5) / 4
p_value = 1 - stats.f.cdf (np.abs (delta), 4, N - 5)

print ("p-value коэффициента детерминации:", p_value)

if (p_value > alpha):
    print ("Регрессия не значима.")
else:
    print ("Регрессия значима.")

Коэффициент детерминации: 0.8106060606060606
p-value коэффициента детерминации: 5.407435041959729e-07
Регрессия значима.


### b) Провести попарное сравнение средних

In [34]:
betta = regression [0]
p_value_pairs = np.zeros (int (len (IgA) * (len (IgA) - 1) / 2))
k = 0

for i in range (len (IgA)):
    for j in range (i + 1, len (IgA)):
        
        delta_up = (betta[i] - betta[j]) * np.sqrt (N - len(IgA))
        delta_down = np.sqrt (RSS * (F_inverse[i][i] + F_inverse[j][j]))
        
        delta = delta_up / delta_down
        p_value = 2 * (1 - stats.t.cdf (np.abs (delta), N - len (IgA)))

        p_value_pairs [k] = p_value
        k += 1
        
        print ("Сравнение {i} и {j} - p-value = {p_value}".format (i=i, j=j, p_value=p_value))
        
        if (p_value > alpha):
            print ("Не похожи.")
        else:
            print ("Похожи.")

Сравнение 0 и 1 - p-value = 0.10310040890082384
Не похожи.
Сравнение 0 и 2 - p-value = 0.0001662014487755492
Похожи.
Сравнение 0 и 3 - p-value = 2.782111500732043e-06
Похожи.
Сравнение 0 и 4 - p-value = 2.4125702768884594e-06
Похожи.
Сравнение 1 и 2 - p-value = 0.00039504010033408754
Похожи.
Сравнение 1 и 3 - p-value = 2.5534724907849693e-06
Похожи.
Сравнение 1 и 4 - p-value = 4.085183105129175e-06
Похожи.
Сравнение 2 и 3 - p-value = 0.0023933387043622023
Похожи.
Сравнение 2 и 4 - p-value = 0.0010025629060046448
Похожи.
Сравнение 3 и 4 - p-value = 0.29579135227474707
Не похожи.


In [37]:
p_value_pairs.sort()
print ("0-4, 1-3, 0-3, 1-4, 0-2, 1-2, 2-4, 2-3, 0-1, 3-4")
k = 10
for i in range (10):
    if (p_value_pairs[i] / k > alpha):
        print ("Не похожи.")
    else:
        print ("Похожи.")
    k -= 1

0-4, 1-3, 0-3, 1-4, 0-2, 1-2, 2-4, 2-3, 0-1, 3-4
Похожи.
Похожи.
Похожи.
Похожи.
Похожи.
Похожи.
Похожи.
Похожи.
Не похожи.
Не похожи.
